In [3]:
import pandas as pd
import numpy as np

TRADES_FILE = "./13-trading_output_regression_insp500_spyfilter_cap15/13-trades_regression_insp500_spyfilter_cap15.parquet"
EQUITY_FILE = "./13-trading_output_regression_insp500_spyfilter_cap15/13-equity_curve_regression_insp500_spyfilter_cap15.parquet"

LOW_CASH_FLOOR = 2000.0
REPORT_DAY = "Wednesday"

# ------------------------------------------------------------
# LOAD DATA
# ------------------------------------------------------------

trades = pd.read_parquet(TRADES_FILE)
trades["exec_date"] = pd.to_datetime(trades["exec_date"])
trades = trades.sort_values("exec_date")

eq = pd.read_parquet(EQUITY_FILE)
eq["date"] = pd.to_datetime(eq["date"])
eq = eq.sort_values("date").set_index("date")

# Group trades by execution date
trades_by_date = {d: g for d, g in trades.groupby("exec_date")}

# ------------------------------------------------------------
# WEEKLY DATES (FROM EQUITY CURVE — SOURCE OF TRUTH)
# ------------------------------------------------------------

weekly_dates = [
    d for d in eq.index
    if d.day_name() == REPORT_DAY
]

records = []

for dt in weekly_dates:

    # -------------------------
    # CASH (AUDITED)
    # -------------------------
    if dt in trades_by_date:
        day_trades = trades_by_date[dt]
        cash_before = float(day_trades["cash_before"].iloc[0])
        cash_after  = float(day_trades["cash_after"].iloc[-1])
        cash_delta  = cash_after - cash_before
    else:
        # No trades → use equity curve
        cash_before = cash_after = float(eq.loc[dt, "cash"])
        cash_delta  = 0.0

    # -------------------------
    # PORTFOLIO VALUES
    # -------------------------
    portfolio_value = float(eq.loc[dt, "portfolio_value"])
    securities_value = portfolio_value - cash_after

    records.append({
        "date": dt,
        "portfolio_value": portfolio_value,
        "cash_before": cash_before,
        "cash_after": cash_after,
        "cash_delta": cash_delta,
        "securities_value": securities_value,
        "num_positions": int(eq.loc[dt, "num_positions"]),
        "low_cash_flag": cash_after < LOW_CASH_FLOOR
    })

# ------------------------------------------------------------
# OUTPUT
# ------------------------------------------------------------

weekly_df = pd.DataFrame(records).set_index("date")

print("\n=== WEEKLY CASH REPORT (AUDITED, AUTHORITATIVE) ===\n")
for dt, r in weekly_df.iterrows():
    flag = " **LOW CASH**" if r["low_cash_flag"] else ""
    print(
        f"{dt.date()} | "
        f"Portfolio ${r['portfolio_value']:,.2f} | "
        f"Cash ${r['cash_after']:,.2f} | "
        f"Securities ${r['securities_value']:,.2f} | "
        f"Positions {r['num_positions']}{flag}"
    )

weekly_df.to_csv("weekly_cash_report.csv")
print("\nSaved weekly_cash_report.csv")



=== WEEKLY CASH REPORT (AUDITED, AUTHORITATIVE) ===

1999-01-06 | Portfolio $345,000.00 | Cash $345,000.00 | Securities $0.00 | Positions 0
1999-01-13 | Portfolio $345,944.81 | Cash $155,699.38 | Securities $190,245.43 | Positions 8
1999-01-20 | Portfolio $357,141.73 | Cash $182,025.00 | Securities $175,116.73 | Positions 7
1999-01-27 | Portfolio $351,676.32 | Cash $161,557.86 | Securities $190,118.46 | Positions 8
1999-02-03 | Portfolio $363,845.21 | Cash $140,131.84 | Securities $223,713.37 | Positions 8
1999-02-10 | Portfolio $351,366.81 | Cash $189,046.29 | Securities $162,320.52 | Positions 7
1999-02-17 | Portfolio $353,178.89 | Cash $205,577.59 | Securities $147,601.30 | Positions 6
1999-02-24 | Portfolio $363,695.27 | Cash $203,321.44 | Securities $160,373.83 | Positions 6
1999-03-03 | Portfolio $359,637.30 | Cash $223,659.67 | Securities $135,977.63 | Positions 5
1999-03-10 | Portfolio $365,872.14 | Cash $180,914.44 | Securities $184,957.71 | Positions 6
1999-03-17 | Portfolio